In [2]:
from graphviz import Digraph
import os

In [3]:
def precedence(op):
    if op == '*':
        return 3
    elif op == '.':
        return 2
    elif op == '+':
        return 1
    return 0

def is_operator(c):
    return c in {'+', '*', '.'}

def to_postfix(regex):
    def insert_concat(regex):
        result = ""
        for i in range(len(regex)):
            c1 = regex[i]
            result += c1
            if i + 1 < len(regex):
                c2 = regex[i + 1]
                if (c1.isalnum() or c1 == '*' or c1 == ')') and (c2.isalnum() or c2 == '('):
                    result += '.'
        return result

    regex = insert_concat(regex)

    stack = []
    output = ""
    for char in regex:
        if char.isalnum():
            output += char
        elif char == '(':
            stack.append(char)
        elif char == ')':
            while stack and stack[-1] != '(':
                output += stack.pop()
            if not stack:
                raise ValueError("Mismatched parentheses")
            stack.pop()  # remove '('
        elif is_operator(char):
            while stack and stack[-1] != '(' and precedence(char) <= precedence(stack[-1]):
                output += stack.pop()
            stack.append(char)
        else:
            raise ValueError(f"Invalid character in regex: {char}")

    while stack:
        top = stack.pop()
        if top == '(':
            raise ValueError("Mismatched parentheses")
        output += top

    return output

In [4]:
class State:
    def __init__(self):
        self.transitions = {}  # symbol -> set of states
        self.id = None

class NFA:
    def __init__(self, start, end):
        self.start = start
        self.end = end

def build_nfa(postfix):
    state_id = 0
    def new_state():
        nonlocal state_id
        s = State()
        s.id = state_id
        state_id += 1
        return s

    stack = []

    for char in postfix:
        if char.isalnum():
            start = new_state()
            end = new_state()
            start.transitions[char] = {end}
            stack.append(NFA(start, end))
        elif char == '*':
            nfa1 = stack.pop()
            start = new_state()
            end = new_state()
            start.transitions['e'] = {nfa1.start, end}
            nfa1.end.transitions['e'] = {nfa1.start, end}
            stack.append(NFA(start, end))
        elif char == '.':
            nfa2 = stack.pop()
            nfa1 = stack.pop()
            nfa1.end.transitions['e'] = {nfa2.start}
            stack.append(NFA(nfa1.start, nfa2.end))
        elif char == '+':
            nfa2 = stack.pop()
            nfa1 = stack.pop()
            start = new_state()
            end = new_state()
            start.transitions['e'] = {nfa1.start, nfa2.start}
            nfa1.end.transitions['e'] = {end}
            nfa2.end.transitions['e'] = {end}
            stack.append(NFA(start, end))
        else:
            raise ValueError(f"Unknown character in postfix: {char}")

    return stack.pop()

In [5]:
def print_transition_table(nfa):
    visited = set()
    def dfs(state):
        if state.id in visited:
            return []
        visited.add(state.id)
        rows = []
        for symbol, next_states in state.transitions.items():
            for next_state in next_states:
                rows.append((state.id, symbol, next_state.id))
                rows.extend(dfs(next_state))
        return rows

    transitions = dfs(nfa.start)
    print("Transition table:")
    print("_______________")
    print("| from state    | input | to states")
    for t in transitions:
        print(f"| {t[0]:<13} | {t[1]:<5} | {t[2]}")
    print("_______________")
    print(f"Start state: {nfa.start.id}")
    print(f"End state: {nfa.end.id}")
    return transitions, nfa.start.id, nfa.end.id

def visualize_nfa(transitions, start_state, end_state, filename="nfa"):
    dot = Digraph()
    dot.attr(rankdir='LR')

    # Invisible start arrow
    dot.node('start', shape='point')
    dot.edge('start', str(start_state))

    states = set()
    for from_state, symbol, to_state in transitions:
        states.add(from_state)
        states.add(to_state)
        label = 'ε' if symbol == 'e' else symbol
        dot.edge(str(from_state), str(to_state), label=label)

    for s in states:
        if s == end_state:
            dot.node(str(s), shape='doublecircle')
        else:
            dot.node(str(s), shape='circle')

    dot.render(filename, view=True)

In [6]:
def main():
    regex = input("Enter regular expression: ")
    postfix = to_postfix(regex)
    print("Postfix expression:", postfix)
    nfa = build_nfa(postfix)
    transitions, start_state, end_state = print_transition_table(nfa)
    visualize_nfa(transitions, start_state, end_state)

In [9]:
if __name__ == "__main__":
    main()

Postfix expression: ab+*
Transition table:
_______________
| from state    | input | to states
| 6             | e     | 7
| 6             | e     | 4
| 4             | e     | 0
| 0             | a     | 1
| 1             | e     | 5
| 5             | e     | 7
| 5             | e     | 4
| 4             | e     | 2
| 2             | b     | 3
| 3             | e     | 5
_______________
Start state: 6
End state: 7
